In [1]:
import pandas as pd
df_ok = pd.read_csv('filtered_for_classification.csv')
df_ok

,Unnamed: 0,title,source,datetime,return_after_1d,weighted
0,0,Visa Makes Their Grand Entrance Into the NFT A...,Watcher Guru,2021-08-23 20:55:03,-2.255187,-0.578110
1,1,Have you seen our billboard in Times Square to...,Watcher Guru,2021-08-26 03:07:52,-2.880761,-0.614835
2,2,Record breaking sale for Bored Ape,Watcher Guru,2021-08-26 22:14:54,3.342732,0.555237
3,5,Subscribe to our new YouTube Channel:,Watcher Guru,2021-10-09 21:25:19,0.196548,0.416093
4,8,"Join our discord server, big stuff coming soon :",Watcher Guru,2021-11-06 18:40:12,2.596537,0.529590
...,...,...,...,...,...,...
12454,20282,Bitcoin liquidity on weekends is showing signs...,unfolded.,2025-03-28 04:10:37,-3.011766,-0.865066
12455,20283,NY AG files $200 million settlement with Galax...,unfolded.,2025-03-28 09:28:05,-1.801569,-0.797618
12456,20284,Terraform Labs to open crypto creditor claims ...,unfolded.,2025-03-28 15:38:46,-2.315566,-0.690030
12457,20285,FDIC clarifies that banks can engage in permit...,unfolded.,2025-03-28 19:11:54,-1.565353,-0.393629


In [2]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /home/kosmo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kosmo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kosmo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/kosmo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR


# Load Data
df = df_ok[['title', 'weighted']].sample(n=5000, random_state=42)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    #print(' '.join(words))
    return ' '.join(words)

df['cleaned_title'] = df['title'].apply(clean_text)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svr', SVR())
])

param_grid = {
    'tfidf__ngram_range': [(1,7),(1,6),(1,5)],
    'tfidf__max_df': [0.03,0.04,0.05],
    'tfidf__min_df': [1],
    'tfidf__sublinear_tf': [False]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(df['cleaned_title'], df['weighted'])
print(grid_search.best_params_)


{'tfidf__max_df': 0.05, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 7), 'tfidf__sublinear_tf': False}


1. {'tfidf__max_df': 0.1, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 3), 'tfidf__sublinear_tf': False}
2. {'tfidf__max_df': 0.05, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 4), 'tfidf__sublinear_tf': False}
3. {'tfidf__max_df': 0.05, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 5), 'tfidf__sublinear_tf': False}
4. {'tfidf__max_df': 0.05, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 7), 'tfidf__sublinear_tf': False}

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=0.05, min_df=1,ngram_range=(1,7),sublinear_tf=False)),
    ('svr', SVR())
])

param_grid = {
    'svr__C': [0.1], # , 1, 10, 100
    'svr__epsilon': [0.5], #0.01, 0.1, 
    'svr__kernel': ['linear'], # , 'rbf', 'poly'
    'svr__gamma': ['scale'] # , 'auto'
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(df['cleaned_title'], df['weighted'])
print(grid_search.best_params_)

{'svr__C': 0.1, 'svr__epsilon': 0.5, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}


1. {'svr__C': 0.1, 'svr__epsilon': 0.5, 'svr__gamma': 'scale', 'svr__kernel': 'linear'}

In [13]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


df = df_ok[['title', 'weighted']]
df['cleaned_title'] = df['title'].apply(clean_text)

vectorizer =  TfidfVectorizer(max_df=0.05, min_df=1,ngram_range=(1,7),sublinear_tf=False)
x = vectorizer.fit_transform(df['cleaned_title'])
y = df['weighted']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

svm_clf = SVR(kernel='linear', C=0.1, epsilon=0.5, gamma='scale')  
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)



/tmp/ipykernel_103755/474211535.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_title'] = df['title'].apply(clean_text)


In [19]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")


Mean Squared Error (MSE): 0.3462037014601653
Root Mean Squared Error (RMSE): 0.588390772752399
R² Score: 0.0021590190907383455


In [15]:
_, X_demo, _, _ = train_test_split(df['title'], y, test_size=0.2, random_state=42)
df_demo = pd.DataFrame({"title": X_demo, "sentiment": y_test})


In [27]:
hello = df_demo.sort_values('sentiment', ascending=False).head(30)
for _, row in hello.iterrows():  
    print(f"{row['sentiment']} : {row['title']}")

0.984384829551427 : CashApp adds free stock trading
0.957202480238257 : Head of Crypto Capital arrested in connection with money laundering
0.8953299645956553 : Citadel CEO Ken Griffin says he regrets not buying crypto a few years ago.
0.8664774498807996 : $1.5 trillion asset manager Invesco reactivates spot Bitcoin ETF filing.
0.8566796944705687 : Japan's stock market suffers worst losses since 1987.
0.8519517522133754 : Citadel, Fidelity, and Charles Schwab launch new crypto exchange platform, EDX Markets. -
0.8441186125571868 : Since the start of 2023, the volume of BTC traded (excluding Binance) relative to all other cryptocurrencies has increased from 25% to 32% as altcoins suffer particularly steep losses
0.8302115276089562 : Robinhood halts 24 hour trading.• Google : -11%• Amazon : -10%• Microsoft : -9%• NVIDIA : -11%• Apple : -10%• Meta : -10%• Tesla : -10%
0.8227596030289075 : Mysterious 'Robin Hood' hackers donating stolen money
0.8219441223744591 : Nvidia reports $35.1 billi

In [28]:
hello = df_demo.sort_values('sentiment', ascending=True).head(30)
for _, row in hello.iterrows():  
    print(f"{row['sentiment']} : {row['title']}")

-0.968479856992306 : $64 billion asset manager Citadel to offer crypto trading and become a liquidity provider for cryptocurrencies.
-0.9554926265689484 : For the first time BitMEX achieved US $1 billion open interest on the perpetual swap contract
-0.9310079969648656 : JPMorgan CEO Jamie Dimon supports Elon Musk's DOGE efforts, stating that the US government is "inefficient" and "it needs to be done."
-0.9020543962580236 : US House Financial Services says "classifying ETH as a security contradicts previous statements of the SEC and Chair Gensler."
-0.8989956388030269 : (ERC20) balances on exchanges by +125% within the last 30 days
-0.890269822187683 : Australian regulator releases crypto, ICO, and mining guidelines
-0.8799025486758083 : Retail interest in Cardano is “rapidly increasing” according to data from analytics firm IntoTheBlock.
-0.8772971340365349 : Tokyo police arrest two for taking possession of stolen cryptocurrency
-0.8675473206081113 : $340 million liquidated from the c